In [1]:
import pandas as pd 
import numpy as np 
import google.generativeai as genai

c:\Harsh\kya dekhe\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from env import api_key
genai.configure(api_key=api_key)
model = genai.GenerativeModel()

In [3]:
df = pd.read_csv("MoviesOnStreamingPlatforms_updated.csv")
df_cleaned = df.dropna()
df_cleaned.shape
df_cleaned.columns

Index(['ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes', 'Netflix',
       'Hulu', 'Prime Video', 'Disney+', 'Type', 'Directors', 'Genres',
       'Country', 'Language', 'Runtime'],
      dtype='object')

In [4]:
from langchain.prompts import PromptTemplate

# Define the prompt template with an example
prompt_template = """
You are a skilled data analyst and Python expert. You will receive a description of a query, and your task is to generate the equivalent pandas code to query the movie dataset.
Your Aim is to Help the User to find the best movies only.

Description:
{query_description}

Columns:
['ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes', 'Netflix',
       'Hulu', 'Prime Video', 'Disney+', 'Type', 'Directors', 'Genres',
       'Country', 'Language', 'Runtime']

Example:
- If the description is: "Get all movies with an IMDb rating greater than 8."
- The output should be:"IMDb > 8"

Output the query as pandas code ONLY that I can query.
How it will run:
df.query(YOUR_PROVIDED_QUERY)

JUST THE YOUR_PROVIDED_QUERY CODE no OTHER TEXT JUST THE PLAIN QUERY THAT I CAN JUST PLUCK IT IN
"""

# Create the PromptTemplate instance
prompt_template_res = PromptTemplate(
    input_variables=["query_description"],
    template=prompt_template
)



: 

In [16]:
import time
while True:
    print("hi")
    user_input = input("Enter your query: ")

    # First LLM query to generate a pandas query
    res = model.generate_content(contents=prompt_template_res.format(
        query_description=user_input,
    )).text.strip()
    
    print(f"Generated pandas query: {res}")

    # Try to run the pandas query, if it's valid
    try:
        query_result = df.query(res)
        print(query_result)
    except Exception as e:
        print(f"Error executing query: {e}\n Hi")
        print(model.generate_content(contents=f"{user_input}").text.strip())
        continue  # Skip to the next iteration if there's an error

    # Second LLM query for generating a summary based on query results
    summary_prompt = f"""
You have received the results of a pandas query, which provides a selection of movies based on user preferences. 

**Query Result:**
{query_result.to_string(index=False)}

**Task:**
Based on the displayed movies, please offer a relevant recommendation to help the user find a great movie to watch. Consider factors such as genre, popularity, and user ratings in your recommendation.

**User's Description:**
{user_input}

Please present your recommendation clearly and concisely as you are talking to the customer.
"""
    summary_res = model.generate_content(contents=summary_prompt).text.strip()
    
    # Print the recommendation
    print("Recommendation based on query result:")
    print(summary_res)
    
    # Optional: Add a small delay between iterations (useful in case of API rate limits)
    time.sleep(0.5)


hi
Generated pandas query: IMDb > 8
          ID                             Title  Year  Age  IMDb  \
0          1                         Inception  2010  13+   8.8   
1          2                        The Matrix  1999  18+   8.7   
2          3            Avengers: Infinity War  2018  13+   8.5   
3          4                Back to the Future  1985   7+   8.5   
4          5    The Good, the Bad and the Ugly  1966  18+   8.8   
...      ...                               ...   ...  ...   ...   
16441  16442     Newsies: The Broadway Musical  2017  13+   8.5   
16548  16549       The Disney Family Singalong  2020  all   8.1   
16564  16565       Phineas and Ferb: Star Wars  2014  all   8.2   
16582  16583  Phineas and Ferb: Mission Marvel  2013  all   8.3   
16693  16694                         The Flood  2018  NaN   8.1   

      Rotten Tomatoes  Netflix  Hulu  Prime Video  Disney+  Type  \
0                 87%      1.0     0            0        0     0   
1                 87%  